In [5]:
import numpy as np
import time
import random
import string
import sys
import logging
import os
import cv2

import imagehash as ih
import pynput.keyboard as kb
import pynput.mouse as m

from collections import defaultdict
from datetime import datetime
from PIL import ImageGrab, Image
from mss import mss
from datetime import datetime
from glob import glob

In [6]:
def debug(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)


def same_images(A, B, delta = 5):
    dist = np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))
    return dist < delta

def same_colors(a, b, delta = 10):
    dist = np.linalg.norm(a-b)
    return dist < delta

def color_dist(a, b):
    return np.linalg.norm(a-b)


def _capture_portion_windows(x0, y0, w0, h0):
    x = x0/2 + WIN_OFFSET[0]
    y = y0/2 + WIN_OFFSET[1]
    w = w0/2 + 2
    h = h0/2 + 2

    with mss() as sct:
        monitor = {'left': int(x), 'top': int(y), 'width': int(w), 'height': int(h)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)
    
    img = img.repeat(2,axis=0).repeat(2,axis=1)
    img = img[x0%2:, x0%2:, :]
    img = img[:w0, :h0, :]
    return img

def _capture_portion_mac(x, y, w, h):
    x = x + MAC_OFFSET[0]
    y = y + MAC_OFFSET[1]

    with mss() as sct:
        monitor = {'left': int(x/2), 'top': int(y/2), 'width': int(w/2 + 1), 'height': int(h/2 + 1)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)

    img = img[x%2:, y%2:, :]
    img = img[:w, :h, :]
    return img

def capture_portion(x, y, w, h):
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    if PLATFORM == "windows":
        return _capture_portion_windows(x, y, w, h)
    else:
        return _capture_portion_mac(x, y, w, h)

def capture_screen():
    w, h = (1200, 900)
    return capture_portion(0, 0, w, h)

def capture_whole_screen():
    with mss() as sct:
        monitor = sct.monitors[1]
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)
    
    if PLATFORM == "windows":
        img = img.repeat(2,axis=0).repeat(2,axis=1)

    return img

def capture_pixel(x, y):
    return capture_portion(x, y, 1, 1)[0,0,:]

def show_image(arr, title = None):
    img = numpy_to_image(arr)
    if title is not None:
        img.show(title = title)
    else:
        img.show()

def save_image(arr, fname=None):
    if fname is None:
        fname = "saved/{}.png".format(time.time())
    img = numpy_to_image(arr)
    img.save(fname)


def numpy_to_image(arr):
    return Image.fromarray(arr.swapaxes(0, 1).astype(np.uint8), "RGB")

def image_to_numpy(img):
    return np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]

def image_to_cv2(img):
    img = np.uint8(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def cv2_to_image(img):
    img = img.astype(np.uint8)
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def cv2_to_numpy(img):
    return image_to_numpy(cv2_to_image(img))

def numpy_to_cv2(arr):
    return image_to_cv2(numpy_to_image(arr))


def load_image(fname):
    img = Image.open(fname)
    return image_to_numpy(img)

def save_game_capture(fname=None):
    img = capture_screen()
    if fname is None:
        global game_start
        if game_start is None:
            game_start = datetime.now()
        debug(game_start)
        fname = game_start.strftime(f'saved/%Y-%m-%d %H.%M.%S.png')
    save_image(img, fname)


def calibrate_offset():
    screen = capture_whole_screen()

    screen_top = -1
    screen_bot = -1
    screen_right = -1
    screen_left = -1

    for x in range(0, screen.shape[0], 1208):
        stride = 0
        for y in range(screen.shape[1]):
            if same_colors(screen[x][y], [32, 41, 74], 5):
                stride += 1
            else:
                if stride == 908:
                    screen_top = y - stride + 4
                    screen_bot = y - 4
                elif stride == 4:
                    if screen_top == -1:
                        screen_top = y
                    elif screen_top == y - 904:
                        screen_bot = y - 4
                stride = 0

    for y in range(0, screen.shape[1], 908):
        stride = 0
        for x in range(screen.shape[0]):
            if same_colors(screen[x][y], [32, 41, 74], 5):
                stride += 1
            else:
                if stride == 1208:
                    screen_left = x - stride + 4
                    screen_right = x - 4
                elif stride == 4:
                    if screen_left == -1:
                        screen_left = x
                    elif screen_left == x - 1204:
                        screen_right = x - 4
                stride = 0
    
    debug("screen bounds: ({}, {}) to ({}, {})".format(screen_left, screen_top, screen_right, screen_bot))

    if screen_top != -1 and screen_bot != -1 and screen_right != -1 and screen_left != -1:
        if PLATFORM == "macOS":
            global MAC_OFFSET
            MAC_OFFSET = (screen_left, screen_top)
        else:
            global WIN_OFFSET
            WIN_OFFSET = (int(screen_left)/2, int(screen_top)/2)
    else:
        debug("trouble finding all edges of screen")


def is_game_over_screen():
    pixel = capture_pixel(680, 380)     # piggy bank
    return same_colors(pixel, [246, 196, 195], 10)

def is_arcade_screen():
    pixel = capture_pixel(170, 852)     # magenta arcade cabinet
    return same_colors(pixel, [143, 60, 148], 10)

def is_start_screen():
    pixel = capture_pixel(600, 30)     # red R in logo
    return same_colors(pixel, [235, 52, 35], 10)

def is_great_play_screen():
    pixel = capture_pixel(290, 670)     # dark red gift box
    return same_colors(pixel, [172, 68, 59], 10)

def is_score_exceeds_screen():
    pixel = capture_pixel(330, 465)     # webkinz logo outline
    return same_colors(pixel, [36, 0, 141], 20)

def is_in_game_screen():
    pixel = capture_pixel(1090, 65)     # music note
    return same_colors(pixel, [179, 108, 247], 10)

def is_recess(img = None):
    if img is None:
        pixel = capture_pixel(610,37)
    else:
        pixel = img[610,37,:]
    return same_colors(pixel, [23,46,138])

def get_best_mask(target, masks):
    best_dist = -1
    best_idx = 0
    for i, mask in enumerate(masks):
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_idx = i
    return best_idx, best_dist

def get_best_mask_dict(target, masks):
    best_dist = -1
    best_key = 0
    for key, mask in masks.items():
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_key = key
    return best_key, best_dist

def calc_image_dist(A, B):
    return np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))


def parse_digits(img, dims, pos, num_digits, masks, centered = True):
    w,h = dims
    x,y = pos

    if img is None:
        num_img = capture_portion(x, y, w*num_digits, h)
    else:
        num_img = img[x:x+w*num_digits,y:y+h,:]

    guess_1, error_1 = _parse_digits(num_img, num_digits, masks, dims)

    if not centered:
        return guess_1

    num_digits -= 1
    num_img = num_img[w//2:-w//2,:,:]

    guess_2, error_2 = _parse_digits(num_img, num_digits, masks, dims)

    if error_1 < error_2:
        return guess_1
    else:
        return guess_2

def _parse_digits(num_img, num_digits, masks, dims):
    w,h = dims
    digits = []
    total_error = 0
    for i in range(num_digits):
        digit, error = get_best_mask(num_img[w*i:w*(i+1)], masks)
        if digit == len(masks) - 1:
            digit = ''
        else:
            digit = digit % 10
        digits.append(digit)
        total_error += error
    digits = [x for x in digits if x != '']
    guess = int('0'+''.join(map(str, digits)))
    return guess, total_error

def get_level(img = None):
    dims = 20,30
    num_digits = 3
    masks = score_masks
    centered = True

    if is_recess(img):
        pos = 445,30
        return -1*parse_digits(img, dims, pos, num_digits, masks, centered)
    else:
        pos = 769,30
        return parse_digits(img, dims, pos, num_digits, masks, centered)

def get_time(img = None):
    if not is_recess(img):
        return None

    dims = 20,30
    pos = 753,30
    num_digits = 3
    masks = score_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

def get_money(img = None):
    dims = 20,30
    pos = 972,30
    num_digits = 4
    masks = score_masks
    centered = True

    if is_recess(img):
        pos = 970,30

    return parse_digits(img, dims, pos, num_digits, masks, centered)

def get_sum_goal(img = None):
    dims = 48,70
    pos = 804,173
    num_digits = 3
    masks = sum_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

def get_sum_goal_wait(img = None, sec = 0.05):
    dims = 48,70
    pos = 804,173
    num_digits = 3
    masks = sum_masks
    centered = True

    prev = None
    if img is None:
        while True:
            smol = capture_portion(pos[0],pos[1],dims[0]*num_digits,dims[1]*num_digits)
            if (smol == prev).all():
                break
            prev = smol
            time.sleep(sec)

    return parse_digits(img, dims, pos, num_digits, masks, centered)

def get_sums_left(img = None):
    if not is_recess(img):
        dims = 20,30
        pos = 536,30
        num_digits = 3
        masks = score_masks
        centered = True

        return parse_digits(img, dims, pos, num_digits, masks, centered)

    return None

def get_score(img = None):
    dims = 20,30
    pos = 142,30
    num_digits = 7
    masks = score_masks
    centered = True

    return parse_digits(img, dims, pos, num_digits, masks, centered)

def press_key(key):
    keyboard.press(key)
    time.sleep(random.uniform(0.1, 0.15))
    keyboard.release(key)
    time.sleep(random.uniform(0.1, 0.15))
    
def click_mouse(x, y):
    if PLATFORM == "macOS":
        x = (x + MAC_OFFSET[0])/2
        y = (y + MAC_OFFSET[1])/2
    else:
        x = x + WIN_OFFSET[0]
        y = y + WIN_OFFSET[1]
    mouse.position = (x, y)
    # time.sleep(random.uniform(0.2, 0.5))
    # mouse.press(m.Button.left)
    # time.sleep(random.uniform(0.1, 0.15))
    # mouse.release(m.Button.left)
    # time.sleep(random.uniform(0.1, 0.15))
    time.sleep(random.uniform(0.025, 0.05))
    mouse.press(m.Button.left)
    time.sleep(random.uniform(0.025, 0.05))
    mouse.release(m.Button.left)
    time.sleep(random.uniform(0.025, 0.05))

def capture_tile(idx, img=None):
    x,y = 40,116
    w,h = 62,62
    a,b = 8,8
    M,N = 10,11
    if img is not None:
        tile = img[x+(w+a)*idx[0]:x+(w+a)*idx[0]+w, y+(y+b)*idx[1]:y+(y+b)*idx[1]+h, :]
    else:
        tile = capture_portion(x+(w+a)*idx[0], y+(y+b)*idx[1], w, h)
    return tile

def click_tile(idx):
    corner = np.array([40,116])
    dims = np.array([62,62])
    offset = np.array([8,8])
    pos = corner + 0.5*dims + (dims+offset)*idx
    click_mouse(pos[0], pos[1])

def subtract(A, B):
    img = np.stack((np.any(np.not_equal(A, B), axis=2),)*3, axis=-1)*A
    return img

def avg_color(img):
    return np.mean(img, axis=(0, 1))

def decode_board(img=None):
    if img is None:
        img = capture_screen()
    board = np.empty(shape=[10, 11], dtype=object)
    for i in range(10):
        for j in range(11):
            tile = decode_tile(img=img, idx=np.array([i,j]))
            board[i,j] = tile
    return board

def capture_board_wait(sec=0.01):
    x,y = 40,116
    w,h = 62,62
    a,b = 8,8
    M,N = 10,11

    prev_img = None
    prev = None
    start = time.time()
    while True:
        time.sleep(sec)
        board_img = capture_screen()
        board = remove_floating(decode_board(board_img))
        if (board_img==prev_img).all() or (board == prev).all():
            break
        prev_img = board_img
        prev = board
    return board_img

def deselect_selected(board):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            tile = board[i,j]
            if "true" in tile:
                click_tile(np.array([i,j]))
            board[i][j] = tile.replace("true", "false")
    return board

def get_dominant_color(arr):
    img = numpy_to_image(arr)
    img.convert("RGB")
    img.resize((1, 1), resample=0)
    arr = image_to_numpy(img)
    return arr[0,0,:]

def print_board(board):
    global STICKER_FGS
    global TILE_COLORS
    global TILE_SYM

    output = ""
    for j in range(0, board.shape[1]):
        for i in range(0, board.shape[0]):
            tile = board[i,j]
            if tile not in SYMBOLS:
                number, sticker, selected = tile.split("_")
                if selected.lower() == "true":
                    bg = TILE_COLORS["selected"]["bg_code"]
                else:
                    if number.isdigit():
                        num = int(number) % 10
                        if num == 0:
                            num = 10
                        bg = TILE_COLORS[str(num)]["bg_code"]
                    else:
                        bg = TILE_COLORS[number]["bg_code"]
                fg = STICKER_FGS[sticker]["code"]
                sym = TILE_SYM[number]
                SYMBOLS[tile] = f"\033[38;5;{fg}m\033[48;5;{bg}m{sym}\033[0m"
            output += SYMBOLS[tile]
        output += "\n"
    print(output)
    print()

# TODO: write this somewhere so don't have to calc each time
def update_level_info(level):
    diff = subtract(capture_screen(), bg)
    diff = diff[:,100:,:]   # ignore first 100 lines
    img = np.any(np.not_equal(diff, np.zeros([3])), axis=-1)
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    x,y,w,h = ymin, xmin+100, ymax-ymin+1, xmax-xmin+1
    level_offsets[level-1] = np.array([x,y])
    level_board[level-1] = np.array([w,h])

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]], count[order[::-1]]

def my_color_hash(img):
    colors, freqs = palette(img)
    hsh = set()
    i = 0
    num_colors = 3
    while len(hsh) < num_colors:
        if i > colors.shape[0] - 1:
            break
        else:
            color = colors[i,:]
            freq = freqs[i] / (img.shape[0]*img.shape[1])
            if freq < 0.02:
                i += 1
                continue
            # if (color==np.array([121,207,249])).all() or (color==np.array([0,25,214])).all():  # ignore light & dark blue
            if (color==np.array([0,25,214])).all():  # ignore dark blue
                i += 1
                continue
            hsh.add("%02x%02x%02x" % (colors[i,0], colors[i,1], colors[i,2]))
            i += 1
    return frozenset(hsh)

def test_number_parser():
    digit_tests = [
        { 'path': 'saved/2021-09-05 19.41.00.png', 'expected': {'score': 4, 'sums': 19, 'level': 1, 'money': 0, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.07.png', 'expected': {'score': 8, 'sums': 18, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.12.png', 'expected': {'score': 12, 'sums': 17, 'level': 1, 'money': 0, 'sum': 7, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.18.png', 'expected': {'score': 18, 'sums': 16, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-07 11.29.03.png', 'expected': {'score': 260, 'sums': None, 'level': -1, 'money': 17, 'sum': 7, 'time': 10} },
        { 'path': 'saved/2021-09-05 19.41.23.png', 'expected': {'score': 22, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.37.png', 'expected': {'score': 38, 'sums': 11, 'level': 1, 'money': 2, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.45.png', 'expected': {'score': 50, 'sums': 9, 'level': 1, 'money': 3, 'sum': 3, 'time': None} },
        { 'path': 'saved/2021-09-05 19.41.52.png', 'expected': {'score': 56, 'sums': 7, 'level': 1, 'money': 3, 'sum': 7, 'time': None} },
        { 'path': 'saved/2021-09-05 19.42.04.png', 'expected': {'score': 72, 'sums': 3, 'level': 1, 'money': 4, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-05 19.42.15.png', 'expected': {'score': 72, 'sums': 3, 'level': 1, 'money': 4, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-05 19.42.31.png', 'expected': {'score': 84, 'sums': 18, 'level': 2, 'money': 5, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.42.36.png', 'expected': {'score': 84, 'sums': 18, 'level': 2, 'money': 5, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.42.49.png', 'expected': {'score': 91, 'sums': 17, 'level': 2, 'money': 6, 'sum': 7, 'time': None} },
        { 'path': 'saved/2021-09-05 19.43.08.png', 'expected': {'score': 126, 'sums': 9, 'level': 2, 'money': 8, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.43.15.png', 'expected': {'score': 131, 'sums': 8, 'level': 2, 'money': 8, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-05 19.43.41.png', 'expected': {'score': 154, 'sums': 20, 'level': 3, 'money': 10, 'sum': 5, 'time': None} },
        { 'path': 'saved/2021-09-05 19.43.48.png', 'expected': {'score': 158, 'sums': 19, 'level': 3, 'money': 10, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.43.54.png', 'expected': {'score': 158, 'sums': 19, 'level': 3, 'money': 10, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 19.44.01.png', 'expected': {'score': 164, 'sums': 18, 'level': 3, 'money': 10, 'sum': 8, 'time': None} },
        { 'path': 'saved/2021-09-05 19.44.21.png', 'expected': {'score': 168, 'sums': 17, 'level': 3, 'money': 11, 'sum': 8, 'time': None} },
        { 'path': 'saved/2021-09-05 20.50.24.png', 'expected': {'score': 463, 'sums': 20, 'level': 5, 'money': 30, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-05 20.52.03.png', 'expected': {'score': 780, 'sums': 20, 'level': 7, 'money': 52, 'sum': 9, 'time': None} },
        { 'path': 'saved/2021-09-05 20.52.12.png', 'expected': {'score': 795, 'sums': 17, 'level': 7, 'money': 53, 'sum': 10, 'time': None} },
        { 'path': 'saved/2021-09-06 10.26.40.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-06 10.26.44.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-06 10.27.02.png', 'expected': {'score': 34, 'sums': 14, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-06 10.27.22.png', 'expected': {'score': 18, 'sums': 16, 'level': 1, 'money': 1, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-06 10.27.45.png', 'expected': {'score': 31, 'sums': 15, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-06 10.27.53.png', 'expected': {'score': 41, 'sums': 13, 'level': 1, 'money': 2, 'sum': 4, 'time': None} },
        { 'path': 'saved/2021-09-06 10.28.13.png', 'expected': {'score': 28, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-06 10.28.18.png', 'expected': {'score': 28, 'sums': 15, 'level': 1, 'money': 1, 'sum': 6, 'time': None} },
        { 'path': 'saved/2021-09-06 13.35.38.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 3, 'time': None} },
        { 'path': 'saved/2021-09-06 13.35.47.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 0, 'time': None} },
        { 'path': 'saved/2021-09-06 13.56.36.png', 'expected': {'score': 1293, 'sums': 20, 'level': 9, 'money': 96, 'sum': 11, 'time': None} },
        { 'path': 'saved/2021-09-07 11.27.09.png', 'expected': {'score': 0, 'sums': 20, 'level': 1, 'money': 0, 'sum': 3, 'time': None} },
        { 'path': 'saved/2021-09-07 11.29.03.png', 'expected': {'score': 260, 'sums': None, 'level': -1, 'money': 17, 'sum': 7, 'time': 10} }
    ]

    for test in digit_tests:
        img = load_image(test['path'])
        actual = {'score': get_score(img), 'sums': get_sums_left(img), 'level': get_level(img), 'money': get_money(img), 'sum': get_sum_goal(img), 'time': get_time(img)}
        if actual != test['expected']:
            print(f"Failed test #{i} ({test['path']}):\n\tACTUAL:\t\t{actual}\n\tEXPECTED:\t{test['expected']}")

def decode_sticker(tile_img):
    if same_colors(tile_img[39,13], np.array([132,148,68])) and same_colors(tile_img[33,53], np.array([196,41,43])):
        name = "apple"
    elif same_colors(tile_img[12,39], np.array([244,189,193])) and same_colors(tile_img[52,24], np.array([237,112,120])):
        name = "eraser"
    elif same_colors(tile_img[31,10], np.array([100,24,145])) and same_colors(tile_img[31,50], np.array([65,141,172])):
        name = "clock"
    else:
        name = "none"
    return name

def decode_type(tile):
    best_name = None
    best_dist = 1000
    outline_pix = tile[30,1]
    for name, colors in TILE_COLORS.items():
        if "outline" in colors:
            if (dist := color_dist(colors["outline"], outline_pix)) < best_dist:
                best_dist = dist
                best_name = name
    # if best_dist > 10:
    #     print(f"WARNING COLOR DIST: {outline_pix} {TILE_COLORS[name]['outline']} {best_dist} {best_name}")
    color_1 = best_name

    best_name = None
    best_dist = 1000
    outline_pix = tile[30,60]
    for name, colors in TILE_COLORS.items():
        if "outline" in colors:
            if (dist := color_dist(colors["outline"], outline_pix)) < best_dist:
                best_dist = dist
                best_name = name
    # if best_dist > 10:
    #     print(f"WARNING COLOR DIST: {outline_pix} {TILE_COLORS[name]['outline']} {best_dist} {best_name}")
    color_2 = best_name

    if "banner" in [color_1, color_2]:
        return "blank"

    if (color_1 == "10" or color_1 == "blank") and (color_2 == "10" or color_2 == "blank"):
        if same_colors(tile[14,22], np.array([255]*3)):
            return "10"

    if color_1 == color_2:
        if color_1 != "blank" and same_colors(tile[14,22], np.array([255]*3)):
            return int(color_1) + 10
        return color_1
    else:
        return "blank"

def decode_tile(img = None, idx = None):
    if img is None:
        img = capture_screen()
    if idx is None:
        idx = np.array([0,0])
    x,y = 40,116
    w,h = 62,62
    a,b = 8,8
    M,N = 10,11
    tile = img[x+idx[0]*(w+a):x+idx[0]*(w+a)+w, y+idx[1]*(h+b):y+idx[1]*(h+b)+h, :]
    name = decode_type(tile)
    selected = same_colors(tile[10,10,:], TILE_COLORS["selected"]["fill"])
    sticker = decode_sticker(tile)
    if name == "blank":
        selected = False
        sticker = "none"
    return f"{name}_{sticker}_{str(selected).lower()}"

def remove_floating(board):
    for i in range(board.shape[0]):
        for j in range(board.shape[1]-2,-1,-1):
            if "blank" in board[i][j+1]:
                name, sticker, selected = board[i][j].split("_")
                board[i][j] = f"blank_none_false"
    return board

# Level 
def tile_priority(tile):
    name, sticker, selected, col, top_idx = tile.split("_")
    if name == "blank":
        name = "100"
    return int(top_idx), sticker == "none", int(name), bool(selected), col

# Level 13, 4080 (w/ if press < 7)
# Level 13, 4150 (w/ while press < 9-level/2.5)
def tile_priority_original(tile):
    name, sticker, selected, col, top_idx = tile.split("_")
    if name == "blank":
        name = "100"
    return int(top_idx), sticker == "none", int(name), bool(selected), col

# Level 13, 2729 (w/ press < 7)
def tile_priority_unused(tile):
    name, sticker, selected, col, top_idx = tile.split("_")
    if name == "blank":
        name = "100"
    return sticker == "none", int(top_idx), bool(selected)

# # Notes: smaller first makes sense
# # Level 12, 3569 (w/ while 9-l//2.5)
# def tile_priority(tile):
#     name, sticker, selected, col, top_idx = tile.split("_")
#     if name == "blank":
#         name = "100"
#     return sticker == "none", int(top_idx), int(name), bool(selected), col


def get_tile_freqs(board):
    tile_freqs = defaultdict(int)
    for i in range(board.shape[0]):
        top_idx = None
        for j in range(board.shape[1]):
            tile = board[i][j]
            if "blank" not in tile:
                if top_idx is None:
                    top_idx = j
                tile += f"_{i}_{top_idx}"
                tile_freqs[tile] += 1
    tile_freqs = {key: tile_freqs[key] for key in sorted(tile_freqs.keys(), key=tile_priority)}
    return tile_freqs

def pick_tiles(tile_freqs, goal):
    if goal < 0:
        return False, {}
    if goal == 0:
        return True, defaultdict(int)
    for i, tile_freq in enumerate(tile_freqs.items()):
        tile,freq = tile_freq
        name, sticker, selected, col, top_idx = tile.split("_")
        if freq == 0:
            continue
        val = int(name)

        new_freqs = defaultdict(int)
        for j, new_freq in enumerate(tile_freqs.items()):
            ti,fr = new_freq
            na,st,se,co,to = ti.split("_")
            if j >= i and fr > 0:
                if co == col:
                    to = str(int(to) + 1)
                if ti == tile:
                    new_freqs["_".join([na,st,se,co,to])] = fr - 1
                else:
                    new_freqs["_".join([na,st,se,co,to])] = fr
        new_freqs = {key: new_freqs[key] for key in sorted(new_freqs.keys(), key=tile_priority)}

        possible, ans = pick_tiles(new_freqs, goal - val)
        if possible:
            ans[tile] += 1
            return possible, ans
    return False, {}

def find_best_tiles(picked, board):
    new_picked = defaultdict(int)
    for k,v in picked.items():
        name, sticker, selected, col, top_idx = k.split("_")
        new_picked["_".join([name, sticker, selected, col])] += v
    tiles = []
    for j in range(board.shape[1]):
        for i in range(board.shape[0]):
            tile = board[i][j]
            tile += f"_{i}"
            if tile in new_picked:
                new_picked[tile] -= 1
                if new_picked[tile] == 0:
                    new_picked.pop(tile)
                tiles.append(np.array([i,j]))
    return tiles

def submit_sum(tiles):
    for tile in tiles:
        i,j = tile[0], tile[1]
        click_tile(tile)

def calc_max_height(board):
    for j in range(board.shape[1]):
        for i in range(board.shape[0]):
            tile = board[i][j]
            if "blank" not in tile:
                return board.shape[1] - j
    return 0
            
def on_press(key):
    global break_program
    if key == kb.Key.esc:
        print("escape pressed")
        break_program = True
        return False
    if key == kb.Key.tab:
        print("tab pressed")
        save_game_capture()

def log_level_info(msg = "MESSAGE", details = "details"):
    global prev_score, prev_time, prev_level, prev_money, logger
    img = capture_screen()
    curr_score, curr_time, curr_level, curr_money = get_score(img), get_time(img), get_level(img), get_money(img)
    score_inc, time_inc, level_inc, money_inc = curr_score - prev_score, curr_time - prev_time, curr_level - prev_level, curr_money - prev_money
    prev_score, prev_time, prev_level, prev_money = curr_score, curr_time, curr_level, curr_money
    logger.debug("\t".join([str(x) for x in [msg, details, level, curr_score, curr_time, curr_level, curr_money, score_inc, time_inc, level_inc, money_inc]]))
    return curr_score, curr_time, curr_level, curr_money


In [7]:
verbose = len(sys.argv) > 1 and sys.argv[1] == "-v"

# macOS, use native values rgb picker
# macOS, Desktop app: 16" MBP 2019, resolution scaled more space resolution
MAC_OFFSET = (1081, 675)   
MAC_DIMS = (1200, 900)
# windows: Microsoft edge, win+<-, my desktop, monitor (1920x1080)
WIN_OFFSET = (389, 230)
WIN_DIMS = (600, 450)

if os.name == "nt":
    PLATFORM = "windows"
elif os.name == "posix":
    PLATFORM = "macOS"

EMOJIS = None
SYMBOLS = None
BACKGROUNDS = None

keyboard = kb.Controller()
mouse = m.Controller()

game_start = None
# calibrate_offset()
SYMBOLS = {}

TILE_SYM = {f"{i}": f"{i:>2}" for i in range(1,100)}
TILE_SYM["blank"] = "  "
TILE_SYM["banner"] = "  "

STICKER_FGS = {
    "clock": {"code": 27},
    "eraser": {"code": 213},
    "apple": {"code": 124},
    "none": {"code": 15}
}

# Note: 8+ are estimates
TILE_COLORS = {
    "selected": {"fill": np.array([153,153,153]), "bg_code": 246},
    "blank": {"fill": np.array([39,92,23]), "outline": np.array([39,92,23]), "bg_code": 22},
    "banner": {"fill": np.array([73,162,248]), "outline": np.array([43,104,168]), "bg_code": 39},
    "1": {"fill": np.array([254,245,81]), "outline": np.array([230,214,71]), "bg_code": 220},
    "2": {"fill": np.array([237,123,47]), "outline": np.array([218,103,41]), "bg_code": 202},
    "3": {"fill": np.array([235,52,35]), "outline": np.array([147,27,19]), "bg_code": 196},
    "4": {"fill": np.array([164,31,22]), "outline": np.array([95,16,10]), "bg_code": 88},
    "5": {"fill": np.array([222,58,246]), "outline": np.array([130,29,148]), "bg_code": 201},
    "6": {"fill": np.array([125,33,186]), "outline": np.array([73,19,82]), "bg_code": 91},
    "7": {"fill": np.array([89,199,250]), "outline": np.array([65,147,179]), "bg_code": 45},
    "8": {"fill": np.array([8,23,149]), "outline": np.array([1,4,73]), "bg_code": 19},
    "9": {"fill": np.array([108,229,70]), "outline": np.array([56,129,35]), "bg_code": 76},
    "10": {"fill": np.array([64,139,39]), "outline": np.array([39,94,24]), "bg_code": 28},
}

sum_masks = []
for i in range(10):
    img = Image.open('sum/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    sum_masks.append(arr)
img = Image.open('sum/mask_blank.png')
arr = image_to_numpy(img)
sum_masks.append(arr)

score_masks = []
for i in range(10):
    img = Image.open('score/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    score_masks.append(arr)
img = Image.open('score/mask_B.png')
arr = image_to_numpy(img)
score_masks.append(arr)
img = Image.open('score/mask_blank.png')
arr = image_to_numpy(img)
score_masks.append(arr)

# bg = load_image("background.png")

break_program = False

listener = kb.Listener(on_press=on_press)
listener.start()

# test = load_image("saved/2021-09-05 19.43.48.png")
# # test = load_image("saved/2021-09-06 13.56.36.png")
# board = decode_board(img=test)
# print_board(board)
# board = remove_floating(board)
# print_board(board)

# tile_freqs = get_tile_freqs(board)
# print(tile_freqs)

# goal = get_sum_goal(test)
# print(goal)
# picked = pick_tiles(tile_freqs, goal)
# print(picked[0], picked[1])
# if picked[0]:
#     best = find_best_tiles(picked[1], board)
#     print(best)
#     # submit_sum(best)
# else:
#     print("Sum not possible with current board")

# exit(1)

click_mouse(0,0)

known_goal = None
presses = 0

# TODO: could perhaps avoid re-decoding board during recess

while not break_program:
    # prev_score, prev_time, prev_level, prev_money = 0,0,0,0
    game_over = False

    if is_score_exceeds_screen():
        click_mouse(670, 520)
        time.sleep(0.5)
    if is_great_play_screen():
        click_mouse(470, 980)
        time.sleep(0.5)
    if is_game_over_screen():
        click_mouse(250, 700)
        time.sleep(0.3)
    if is_start_screen():
        click_mouse(180, 850)
        time.sleep(0.2)
    if not is_in_game_screen():
        print("Something is rotten in denmark")
        exit(1)
    capture_board_wait()

    # level = get_level()
    # log_level_info("NEW GAME", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))

    while not break_program and not game_over:
        # print(f"LEVEL: {level}")
        # update_level_info(level)

        while not break_program:

            board = decode_board()
            # print_board(board)
            board = remove_floating(board)
            board = deselect_selected(board)
            print_board(board)

            if not is_recess():
                while (max_height := calc_max_height(board)) + presses < 9 - int(get_level() / 2.5):
                    click_mouse(700, 145)
                    presses += 1
            presses = 0

            tile_freqs = get_tile_freqs(board)
            # print(tile_freqs)

            if not is_recess():
                goal = get_sum_goal_wait()
                known_goal = None
            elif known_goal is None or known_goal == 0:
                goal = get_sum_goal_wait()
                known_goal = goal

            picked = pick_tiles(tile_freqs, goal)
            print(goal, len(picked[1]))
            if picked[0]:
                best = find_best_tiles(picked[1], board)
                print(best)
                submit_sum(best)
                capture_board_wait()
            else:
                print("Sum not possible with current board")
                click_mouse(700, 145)
                capture_board_wait()

                    
                    
                    
                    
                    
                    
                    
 3 2 2 1 2 3 1 1 2 3
 1 2 2 2 2 1 1 2 3 3
 1 2 2 1 1 1 2 2 1 2
 2 2 3 2 2 1 2 2 2 3


6 6
[array([3, 7]), array([6, 7]), array([7, 7]), array([0, 8]), array([5, 8]), array([4, 9])]
                    
                    
                    
                    
   2 3           1 3
 3 2 1 3 3 3 3 2 1 1
 2 2 2 3 3 3 3 3 3 2
 3 2 2 2 2 2 2 2 2 3
 3 2 2 2 2 3 1 2 3 3
 1 2 2 1 2 1 2 2 1 2
 2 2 3 2 2 1 2 2 2 3


3 2
[array([1, 5]), array([2, 5])]
                    
                    
                    
                 1 2
 1 3 3 2 3 1 3 2 1 3
 3 2 3 3 3 3 3 2 1 1
 2 2 2 3 3 3 3 3 3 2
 3 2 2 2 2 2 2 2 2 3
 3 2 2 2 2 3 1 2 3 3
 1 2 2 1 2 1 2 2 1 2
 2 2 3 2 2 1 2 2 2 3


7 5
[array([8, 3]), array([0, 4]), array([3, 4]), array([9, 5]), array([5, 7])]
                    
                    
                    
                    
   2 1 